In [1]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten
from keras.optimizers import Adam
import gym
from rl.agents.dqn import DQNAgent
from rl.policy import EpsGreedyQPolicy
from rl.memory import SequentialMemory
import json
import numpy as np

Using TensorFlow backend.


### Agent and Environment Description

* observation:  
    (Cart Position ,  Cart Velocity ,  Pole Angle ,  Pole Velocity At Tip)
 
* Actions:  
    0 Push cart to the left; 1 Push cart to the right

* Reward:  
    Reward is for every step taken ,  including the termination step

* Starting State:  
    All observations are assigned a uniform random value between +-0.05

* Episode Termination:  
    Pole Angle is more than ±12°
    Cart Position is more than ± 2.4 
    (center of the cart reaches the edge of the display)
    Episode length is greater than 200

* reference:  
    http:  //neuro-educator.com/rl1/


In [2]:
env = gym.make('CartPole-v0')

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.


In [3]:
model = Sequential()
# input: (n x 4) ( Cart Position, Cart Vel, Pole Angle, Pole Vel)
model.add(Flatten(input_shape=(1, env.observation_space.shape[0])))
model.add(Dense(16))
model.add(Activation('relu'))
model.add(Dense(16))
model.add(Activation('relu'))
model.add(Dense(16))
model.add(Activation('relu'))

# output: (n x 3) (left, no, right)
model.add(Dense(env.action_space.n))
model.add(Activation('linear'))

In [4]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 4)                 0         
_________________________________________________________________
dense_1 (Dense)              (None, 16)                80        
_________________________________________________________________
activation_1 (Activation)    (None, 16)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 16)                272       
_________________________________________________________________
activation_2 (Activation)    (None, 16)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 16)                272       
_________________________________________________________________
activation_3 (Activation)    (None, 16)                0         
__________

In [5]:
# memory replay
memory = SequentialMemory(limit=50000, window_length=1)

# epsilon greedy algorithm
policy = EpsGreedyQPolicy(eps=0.001)

dqn = DQNAgent(model=model, nb_actions=env.action_space.n, gamma=0.99, memory=memory,
              nb_steps_warmup=10, target_model_update=1e-2, policy=policy)

In [42]:
dqn.compile(Adam(lr=1e-3), metrics=['mae'])

history = dqn.fit(env, nb_steps=100000, visualize=False, verbose=2)

Training for 100000 steps ...
   200/100000: episode: 1, duration: 1.110s, episode steps: 200, steps per second: 180, episode reward: 200.000, mean reward: 1.000 [1.000, 1.000], mean action: 0.500 [0.000, 1.000], mean observation: 0.091 [-0.553, 0.606], loss: 100.118059, mean_absolute_error: 48.386495, mean_q: 97.471889
   400/100000: episode: 2, duration: 0.410s, episode steps: 200, steps per second: 487, episode reward: 200.000, mean reward: 1.000 [1.000, 1.000], mean action: 0.500 [0.000, 1.000], mean observation: 0.166 [-0.660, 0.952], loss: 21.063047, mean_absolute_error: 49.649826, mean_q: 100.768593
   600/100000: episode: 3, duration: 0.414s, episode steps: 200, steps per second: 483, episode reward: 200.000, mean reward: 1.000 [1.000, 1.000], mean action: 0.500 [0.000, 1.000], mean observation: 0.176 [-0.730, 1.148], loss: 21.213579, mean_absolute_error: 51.815792, mean_q: 105.447777
   800/100000: episode: 4, duration: 0.402s, episode steps: 200, steps per second: 498, episod

  5424/100000: episode: 29, duration: 0.411s, episode steps: 200, steps per second: 487, episode reward: 200.000, mean reward: 1.000 [1.000, 1.000], mean action: 0.500 [0.000, 1.000], mean observation: 0.131 [-0.515, 0.698], loss: 55.919865, mean_absolute_error: 69.482513, mean_q: 139.108704
  5624/100000: episode: 30, duration: 0.400s, episode steps: 200, steps per second: 500, episode reward: 200.000, mean reward: 1.000 [1.000, 1.000], mean action: 0.505 [0.000, 1.000], mean observation: 0.148 [-0.548, 0.747], loss: 39.881207, mean_absolute_error: 69.495979, mean_q: 139.680725
  5824/100000: episode: 31, duration: 0.402s, episode steps: 200, steps per second: 498, episode reward: 200.000, mean reward: 1.000 [1.000, 1.000], mean action: 0.505 [0.000, 1.000], mean observation: 0.170 [-0.629, 0.945], loss: 123.447441, mean_absolute_error: 71.349129, mean_q: 143.637207
  6024/100000: episode: 32, duration: 0.406s, episode steps: 200, steps per second: 493, episode reward: 200.000, mean r

 11024/100000: episode: 57, duration: 0.435s, episode steps: 200, steps per second: 460, episode reward: 200.000, mean reward: 1.000 [1.000, 1.000], mean action: 0.500 [0.000, 1.000], mean observation: 0.423 [-0.563, 2.022], loss: 84.045731, mean_absolute_error: 104.371895, mean_q: 212.004349
 11169/100000: episode: 58, duration: 0.289s, episode steps: 145, steps per second: 502, episode reward: 145.000, mean reward: 1.000 [1.000, 1.000], mean action: 0.510 [0.000, 1.000], mean observation: 0.493 [-0.620, 2.410], loss: 106.162613, mean_absolute_error: 106.787170, mean_q: 217.337601
 11278/100000: episode: 59, duration: 0.225s, episode steps: 109, steps per second: 484, episode reward: 109.000, mean reward: 1.000 [1.000, 1.000], mean action: 0.523 [0.000, 1.000], mean observation: 0.511 [-0.651, 2.406], loss: 106.149384, mean_absolute_error: 109.248268, mean_q: 222.708130
 11420/100000: episode: 60, duration: 0.290s, episode steps: 142, steps per second: 489, episode reward: 142.000, me

 12699/100000: episode: 86, duration: 0.203s, episode steps: 101, steps per second: 498, episode reward: 101.000, mean reward: 1.000 [1.000, 1.000], mean action: 0.554 [0.000, 1.000], mean observation: 0.493 [-0.692, 2.405], loss: 526.536743, mean_absolute_error: 134.398270, mean_q: 270.881439
 12812/100000: episode: 87, duration: 0.242s, episode steps: 113, steps per second: 467, episode reward: 113.000, mean reward: 1.000 [1.000, 1.000], mean action: 0.531 [0.000, 1.000], mean observation: 0.479 [-0.811, 2.414], loss: 237.040482, mean_absolute_error: 135.282913, mean_q: 273.646118
 12940/100000: episode: 88, duration: 0.257s, episode steps: 128, steps per second: 499, episode reward: 128.000, mean reward: 1.000 [1.000, 1.000], mean action: 0.516 [0.000, 1.000], mean observation: 0.461 [-0.730, 2.413], loss: 266.281799, mean_absolute_error: 136.656311, mean_q: 275.865540
 13057/100000: episode: 89, duration: 0.232s, episode steps: 117, steps per second: 504, episode reward: 117.000, m

 15946/100000: episode: 114, duration: 0.173s, episode steps: 81, steps per second: 468, episode reward: 81.000, mean reward: 1.000 [1.000, 1.000], mean action: 0.568 [0.000, 1.000], mean observation: 0.332 [-0.392, 2.005], loss: 245.660477, mean_absolute_error: 142.992950, mean_q: 285.906952
 16067/100000: episode: 115, duration: 0.241s, episode steps: 121, steps per second: 503, episode reward: 121.000, mean reward: 1.000 [1.000, 1.000], mean action: 0.570 [0.000, 1.000], mean observation: 0.458 [-0.620, 3.079], loss: 317.249664, mean_absolute_error: 140.700485, mean_q: 280.654755
 16152/100000: episode: 116, duration: 0.182s, episode steps: 85, steps per second: 467, episode reward: 85.000, mean reward: 1.000 [1.000, 1.000], mean action: 0.565 [0.000, 1.000], mean observation: 0.330 [-0.580, 2.010], loss: 200.886993, mean_absolute_error: 140.842178, mean_q: 282.555115
 16266/100000: episode: 117, duration: 0.228s, episode steps: 114, steps per second: 501, episode reward: 114.000, m

...

 77100/100000: episode: 507, duration: 0.400s, episode steps: 200, steps per second: 500, episode reward: 200.000, mean reward: 1.000 [1.000, 1.000], mean action: 0.490 [0.000, 1.000], mean observation: -0.332 [-2.393, 1.564], loss: 54.594460, mean_absolute_error: 88.647797, mean_q: 177.339172
 77269/100000: episode: 508, duration: 0.378s, episode steps: 169, steps per second: 447, episode reward: 169.000, mean reward: 1.000 [1.000, 1.000], mean action: 0.467 [0.000, 1.000], mean observation: -0.382 [-2.431, 0.959], loss: 59.104080, mean_absolute_error: 88.292969, mean_q: 176.216583
 77445/100000: episode: 509, duration: 0.354s, episode steps: 176, steps per second: 497, episode reward: 176.000, mean reward: 1.000 [1.000, 1.000], mean action: 0.477 [0.000, 1.000], mean observation: -0.311 [-2.630, 1.908], loss: 74.553413, mean_absolute_error: 87.625618, mean_q: 174.461929
 77645/100000: episode: 510, duration: 0.397s, episode steps: 200, steps per second: 504, episode reward: 200.000, 

 82439/100000: episode: 535, duration: 0.412s, episode steps: 200, steps per second: 485, episode reward: 200.000, mean reward: 1.000 [1.000, 1.000], mean action: 0.495 [0.000, 1.000], mean observation: -0.216 [-2.088, 1.545], loss: 39.269455, mean_absolute_error: 70.282562, mean_q: 139.699173
 82639/100000: episode: 536, duration: 0.424s, episode steps: 200, steps per second: 471, episode reward: 200.000, mean reward: 1.000 [1.000, 1.000], mean action: 0.490 [0.000, 1.000], mean observation: -0.373 [-2.114, 1.546], loss: 19.163887, mean_absolute_error: 70.141983, mean_q: 139.949829
 82839/100000: episode: 537, duration: 0.398s, episode steps: 200, steps per second: 502, episode reward: 200.000, mean reward: 1.000 [1.000, 1.000], mean action: 0.490 [0.000, 1.000], mean observation: -0.368 [-2.195, 1.464], loss: 25.301384, mean_absolute_error: 69.694550, mean_q: 139.046936
 83039/100000: episode: 538, duration: 0.427s, episode steps: 200, steps per second: 468, episode reward: 200.000, 

 87878/100000: episode: 563, duration: 0.471s, episode steps: 200, steps per second: 425, episode reward: 200.000, mean reward: 1.000 [1.000, 1.000], mean action: 0.515 [0.000, 1.000], mean observation: 0.074 [-1.891, 1.692], loss: 16.188871, mean_absolute_error: 56.396927, mean_q: 112.349709
 88078/100000: episode: 564, duration: 0.403s, episode steps: 200, steps per second: 496, episode reward: 200.000, mean reward: 1.000 [1.000, 1.000], mean action: 0.490 [0.000, 1.000], mean observation: -0.165 [-1.744, 1.705], loss: 19.748407, mean_absolute_error: 56.080448, mean_q: 111.479919
 88278/100000: episode: 565, duration: 0.424s, episode steps: 200, steps per second: 472, episode reward: 200.000, mean reward: 1.000 [1.000, 1.000], mean action: 0.505 [0.000, 1.000], mean observation: 0.035 [-1.905, 1.732], loss: 14.951132, mean_absolute_error: 55.286453, mean_q: 110.120453
 88478/100000: episode: 566, duration: 0.399s, episode steps: 200, steps per second: 502, episode reward: 200.000, me

 93346/100000: episode: 591, duration: 0.400s, episode steps: 200, steps per second: 499, episode reward: 200.000, mean reward: 1.000 [1.000, 1.000], mean action: 0.495 [0.000, 1.000], mean observation: -0.077 [-0.412, 0.516], loss: 12.400088, mean_absolute_error: 54.233498, mean_q: 108.130783
 93546/100000: episode: 592, duration: 0.480s, episode steps: 200, steps per second: 416, episode reward: 200.000, mean reward: 1.000 [1.000, 1.000], mean action: 0.495 [0.000, 1.000], mean observation: -0.166 [-1.001, 0.450], loss: 11.238359, mean_absolute_error: 53.916222, mean_q: 107.568184
 93746/100000: episode: 593, duration: 0.400s, episode steps: 200, steps per second: 500, episode reward: 200.000, mean reward: 1.000 [1.000, 1.000], mean action: 0.490 [0.000, 1.000], mean observation: -0.240 [-1.498, 0.813], loss: 15.851204, mean_absolute_error: 53.757702, mean_q: 107.091309
 93946/100000: episode: 594, duration: 0.433s, episode steps: 200, steps per second: 462, episode reward: 200.000, 

 98911/100000: episode: 619, duration: 0.401s, episode steps: 200, steps per second: 498, episode reward: 200.000, mean reward: 1.000 [1.000, 1.000], mean action: 0.470 [0.000, 1.000], mean observation: -0.155 [-2.484, 1.812], loss: 10.629272, mean_absolute_error: 47.064945, mean_q: 93.745819
 99111/100000: episode: 620, duration: 0.458s, episode steps: 200, steps per second: 436, episode reward: 200.000, mean reward: 1.000 [1.000, 1.000], mean action: 0.485 [0.000, 1.000], mean observation: -0.333 [-2.163, 0.958], loss: 8.670150, mean_absolute_error: 46.943115, mean_q: 93.641838
 99311/100000: episode: 621, duration: 0.397s, episode steps: 200, steps per second: 504, episode reward: 200.000, mean reward: 1.000 [1.000, 1.000], mean action: 0.490 [0.000, 1.000], mean observation: -0.177 [-1.357, 1.004], loss: 6.864767, mean_absolute_error: 46.699074, mean_q: 93.179535
 99511/100000: episode: 622, duration: 0.416s, episode steps: 200, steps per second: 480, episode reward: 200.000, mean 

In [43]:
with open('cartpole_history.json', 'w') as f:
    json.dump(history.history, f)

dqn.save_weights('cartpole_dqn.hdf5')

[WARNING] cartpole_dqn.hdf5 already exists - overwrite? [y/n]y
[TIP] Next time specify overwrite=True!


### Results

In [52]:
env = gym.make('CartPole-v0')
dqn.load_weights('cartpole_dqn.hdf5')
dqn.test(env, nb_episodes=5, visualize=True)
env.close()

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
Testing for 5 episodes ...
Episode 1: reward: 200.000, steps: 200
Episode 2: reward: 200.000, steps: 200
Episode 3: reward: 200.000, steps: 200
Episode 4: reward: 200.000, steps: 200
Episode 5: reward: 200.000, steps: 200


In [53]:
y = None
with open('cartpole_history.json', 'r') as f:
    y = json.load(f)

In [54]:
import matplotlib.pyplot as plt
%matplotlib inline

In [55]:
fig = plt.figure()
ax = fig.add_subplot(111)
ax.set_title('DQN_rewards')
ax.set_xlabel('episodes')
ax.set_ylabel('rewards')
episode_reward = np.array(y['episode_reward'])
ax.plot(episode_reward)
plt.savefig('CartPole_DQN_rewards')
plt.show()

In [56]:
len(episode_reward)

624

In [61]:
# 1) Avg. reward over the last 100 consecutive episodes
print('Avg. reward: ', np.mean(episode_reward[100:]))

Avg. reward:  163.116412214


In [62]:
# 2) How many episodes before you encounter 100 consecutive episodes
b4 = False
count = 0
max_continuous = 0
for idx, reward in enumerate(episode_reward > 150):
    count = count + 1 if b4 and reward else 0
    max_continuous = max(max_continuous, count)
    b4 = reward

In [63]:
max_continuous  # It doesn't reaches 100

80

### 3) any other informatoins
* The reward for every episodes
* DQN network summery

Are shown abeve.